# Building Machine Learning Classifiers: Explore Gradient Boosting model with grid-search

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

### Read in & clean text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Explore GradientBoostingClassifier Attributes & Hyperparameters

In [2]:
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
print(dir(GradientBoostingClassifier))
print(GradientBoostingClassifier())

['_SUPPORTED_LOSS', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_feature_names', '_check_initialized', '_check_n_features', '_check_params', '_clear_state', '_compute_partial_dependence_recursion', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_get_tags', '_init_state', '_is_initialized', '_make_estimator', '_more_tags', '_raw_predict', '_raw_predict_init', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_resize_state', '_staged_raw_predict', '_validate_data', '_validate_estimator', '_validate_y', 'apply', 

In [4]:
print(GradientBoostingClassifier())

GradientBoostingClassifier()


In [5]:
GradientBoostingClassifier() #press Shift+ Tab for all funct defaults

GradientBoostingClassifier()

In [6]:
#grad boost difference with random forest  ##### 2 snapshots of rf v/s gb and pros and cons of gb
#max_depth of rf was none so each tree is made as deep as possible// in gradboost trees are shorter
#RF has couple of fully grown trees while GB has alot of very basic trees
#gb has learning rate which tells how fast the model learns

### Build our own Grid-search

In [7]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [9]:
def train_GB(est, depth, lr):
    gb = GradientBoostingClassifier(n_estimators=n_est, max_depth=depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred =gb_model.predict(X_test)
    precision, recall, fscore, support= score(y_test,y_pred, pos_label='spam',average='binary')
    print('N_est=',n_est,'Depth',depth,'------>','precision=', precision, 'recall=',recall, 'fscore=',fscore, 'support=',support)
    print('N_est=',n_est,'Depth',depth,'------>',round((y_test==y_pred).sum()/len(y_pred),3))

In [10]:
#This takes too much time
#for n_est in [50, 100, 150]:
#    for depth in [3, 7, 11, 15]:
#        for lr in [0.01, 0.1, 1]:
#            train_GB(n_est, depth, lr)
#first iteration to find out what is best range for each hyper parameter

# next notebook

In [12]:
from sklearn.model_selection import GridSearchCV

In [14]:
gb= GradientBoostingClassifier()
param={
    'n_estimators': [100, 150],
    'max_depth' : [7, 11, 15],
    'learning_rate': [0.1]
}
gs= GridSearchCV(gb,param,cv=5,n_jobs=-1) #(mode, parameters,cross_validation folds, parallel processing)
cv_fit=gs.fit(X_tfidf_feat,data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

NameError: name 'X_tfidf_feat' is not defined